Bunch of code to inspect each ROI's signal compared to a few basic behavioural variables

In [1]:
from vgatPAG.database.db_tables import *
from vgatPAG.analysis.utils import get_mouse_session_data, get_shelter_threat_trips, get_session_stimuli_frames
from vgatPAG.analysis.naiveBayes import NaiveBayes

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.decomposition import PCA
import seaborn as sns
from random import choices
# import statsmodles.api as sm

from brainrender.colors import colorMap

from fcutils.plotting.plot_elements import plot_shaded_withline
from fcutils.plotting.plot_distributions import plot_kde
from fcutils.plotting.colors import *
from fcutils.plotting.utils import save_figure, clean_axes

Connecting root@localhost:3306
settings: screenshots are set to have transparent backgroundset the parameter "SCREENSHOT_TRANSPARENT_BACKGROUND" to false if you would prefer a not transparent background


# Fetching data

Fetching data involves two steps:
1) Get the data for a specific session (experiment) of a specific mouse from the datajoint database
2) Concatenate akk the different data streams from each recording wihtin the session


In [2]:

# Get all mice
mice = Mouse.fetch("mouse")

# Get all sessions
sessions = {m:(Session & f"mouse='{m}'").fetch("sess_name") for m in mice}

# Get the recordings for each session
recordings = {m:{s:(Recording & f"sess_name='{s}'" & f"mouse='{m}'").fetch(as_dict=True) for s in sessions[m]} for m in mice}



In [3]:
for _mouse in mice:
    print(f'\n{_mouse}:')
    for _session in sessions[_mouse]:
        print(f"      |---{_session}")

        recs = Recording().get_sessions_recordings(_mouse, _session)
        print("      |       |--", end='')
        print(*recs, sep=" || ")        



BF161p1:
      |---19JUN03
      |       |--19JUN03_BF161p1_v1 || 19JUN03_BF161p1_v2
      |---19JUN04
      |       |--19JUN04_BF161p1_v1 || 19JUN04_BF161p1_v2

BF164p1:
      |---19JUN05
      |       |--19JUN05_BF164p1_GCaMP_v1
      |---19JUN18
      |       |--19JUN19_BF164p1_GCaMP_t1 || 19JUN19_BF164p1_GCaMP_t2 || 19JUN19_BF164p1_GCaMP_t3_chase

BF164p2:
      |---19JUL15
      |       |--19JUL15_BF164p2_GCaMP_t1 || 19JUL15_BF164p2_GCaMP_t2
      |---19JUN24
      |       |--19JUN24_BF164p2_GCaMP_t2
      |---19JUN26
      |       |--19JUN26_BF164p2_GCaMP_t1

BF166p3:
      |---19JUN05
      |       |--19JUN05_BF166p3_GCaMP_t2 || 19JUN05_BF166p3_GCaMP_t3 || 19JUN05_BF166p3_GCaMP_t4
      |---19JUN19
      |       |--19JUN19_BF166p3_GCaMP_t2 || 19JUN19_BF166p3_GCaMP_t3


# Plotting
## Correlation betweeen position and ROI activity

In [ ]:
# Utility funcs
def get_binned_pc(dflist):
    # Given a list of dataframes with activity binned along one axis,
    # computes PCA and returns the results

    # Get bins
    x_bins = dflist[0].index.values
    int_values = np.array([(i.left + i.right)/2 for i in x_bins])

    # Get signal and remove nans
    binned_signals = np.vstack([df.interpolate().sig.values for df in dflist]).T
    binned_signals[np.isnan(binned_signals)]  = 0

    # Do PCA
    pca = PCA(n_components=1)
    PC = pca.fit_transform(binned_signals)

    # Nan first bins
    PC[(int_values < 250)|(int_values>950)] = np.nan
    
    return int_values, PC


In [ ]:
plt.ioff()

# Params
xbins, ybins = 25, 15
exploration_only = False

# Figure making
for mouse in tqdm(mice):

    all_pcas = []
    for sess in sessions[mouse]:
        x_binned_activity = []
        if exploration_only:
            first_stim = Recording().get_session_first_stim(mouse, sess) 
        else:
            first_stim = True

        if not first_stim: 
            continue

        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess, sessions)

        if len(tracking) != len(signals[0]):
            raise ValueError

        f, axarr = plt.subplots(ncols = 6, nrows=6, figsize=(25, 18))
        axarr = axarr.flatten()
        f.suptitle(f"{mouse} - {sess}")

        for n in np.arange(_nrois):
            # Plot activity projected ont X and Y axis
            trace = pd.DataFrame(dict(
                x = np.int64(tracking['x'].values),
                y = np.int64(tracking['y'].values),
                sig = signals[n],
                isrec = is_rec,
            ))

            if exploration_only: # Keep only up to first stim
                trace = trace[:first_stim]

            if not len(trace): continue

            # ! keep only data for when we are recording
            trace = trace.loc[trace.isrec > 0]
            if not len(trace): continue

            # Plot tracking and binned activity
            axarr[n].set(title=f'ROI {n}')
            axarr[n].hexbin(trace['x'], trace['y'], C=trace.sig, gridsize=xbins, reduce_C_function=np.mean)


            # Bind data along x and y directions
            df = trace.groupby(pd.cut(trace['x'], bins=xbins)).mean()
            int_values = np.array([(i.left + i.right)/2 for i in df.index.values])
            df[int_values < 250] = np.nan
            df[int_values > 950] = np.nan
            plot_shaded_withline(axarr[n], int_values, ((df['sig']/df['sig'].max())*100), z=0, lw=1, color='salmon') # scale the plot to fit nicely
            x_binned_activity.append(df.interpolate())


            df = trace.groupby(pd.cut(trace['y'], bins=ybins)).mean()
            int_values = [(i.left + i.right)/2 for i in df.index.values]
            axarr[n].plot( ((df['sig']/df['sig'].max())*100)+1100, int_values,  lw=4, color='darkseagreen') # scale the plot to fit nicely


            # break
        
        # Plot bin occupancy from tracking
        axarr[n+2].hexbin(tracking['x'], tracking['y'], gridsize=xbins, bins='log', mincnt=1, cmap='inferno')
        _ = axarr[n+2].set(title='Bin occupancy log scale')

        # Plot PCA of activity binned on X axis
        if len(x_binned_activity):
            x_bins, PC = get_binned_pc(x_binned_activity)
            axarr[n+1].plot(x_bins, PC, lw=2, color=darkseagreen) 
            axarr[n+1].set(title='PCA on pop activity binned by X position', xlabel='X position', ylabel='PC1', xlim=[100, 1200])
            all_pcas.append(PC.ravel())

        # Clean axes and save figure
        clean_axes(f)
        f.tight_layout()
        save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\{mouse}_{sess}_ontracking_{"exp" if exploration_only else "all"}', verbose=False)
        plt.close(f)

        # break
    # break

plt.ion()

print('\ndone')

## Plot activity of each ROI binned by X position

In [ ]:
# f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))
f, ax = plt.subplots(figsize=(12, 10))
normalize = True
th = 5

px_to_cm = 13.9

endpoints = [[] for i in np.arange(8)]
for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        # axarr[n, s].set(title=f"{mouse} - {sess}")


        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess, sessions)
        # remove speed outliers
        speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)



        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]

            df = trace.groupby(pd.cut(trace['x'], bins= np.linspace(200, 1100, 9))).mean()
            int_values = np.array([(i.left + i.right)/2 for i in df.index.values])


            y = df.sig

            if normalize:
                y -= y[0]

            if not np.any(np.isnan(y[-1])):
                color = colorMap(y[-1], 'bwr', vmin=-100, vmax=100)
            else:
                color = colorMap(y[-2], 'bwr', vmin=-100, vmax=100)
            ax.plot(int_values, y, 'o-', lw=2, ms=8, mec='k', label=f'roi {nroi}', color=color)
            # axarr[n, s]

            for k, yy in enumerate(y):
                if  not np.any(np.isnan(yy)):
                    endpoints[k].append(yy)


        # axarr[n, s].legend()
        # break
    # break
for k, endp in enumerate(endpoints):
    if k == 0: continue
    plot_kde(data=endp, vertical=True, normto=50, z=int_values[k]+5, ax=ax,  color='gray', zorder=99)


ax.set(title='All ROIs activity binned by speed', xticks=int_values, xticklabels=int_values, xlabel='X position',
                ylabel='Activity', ylim=[-200, 200])

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\allrois_space_modulation', verbose=False)
plt.show()


## Look at correlation between activity and X pos depending on the direction of movement

In [ ]:
for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):
        # Make figure 
        f, axarr = plt.subplots(ncols=6, nrows=6, figsize=(18, 9))
        f.suptitle('RED: positive X speed, BLACK: no speed, BLUE: negative X speed')
        axarr = axarr.flatten()

        # Get tracking
        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess, sessions)
        speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)

        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    x_speed = derivative(tracking.x.values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]

            # Take only times with positive or negative X speed and plot separately 
            dfs = dict(
                positive = (trace.loc[trace.x_speed > 0].reset_index(drop=True), salmon),
                null = (trace.loc[trace.x_speed == 0].reset_index(drop=True), [.4, .4, .4]),
                negative = (trace.loc[trace.x_speed < 0].reset_index(drop=True), lightblue),
            )

            for val, (df, color) in dfs.items():
                df = df.groupby(pd.cut(df['x'], bins=25)).mean()
                int_values = np.array([(i.left + i.right)/2 for i in df.index.values])
                df[int_values < 250] = np.nan
                df[int_values > 950] = np.nan
                axarr[nroi].plot(int_values, df.sig, lw=3, color=color) # scale the plot to fit nicely

            axarr[nroi].set(title=f'ROI - {nroi}')

            
        # Clean axes and save figure
        clean_axes(f)
        f.tight_layout()
        save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\{mouse}_{sess}_activity_vs_X_by_xspeed', verbose=False)
        plt.close(f)

plt.show()

### Keeping only shelter -> threat and threat -> shelter trips

In [ ]:
normalize_by_speed = False # if True the signal will be normalized by the average escape speed in the bin
no_escapes = True # if True it will ignore threat-> shelter trips that happened shortly after a stimulus occurred

speed_th = 4 # Min trip average speed

for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):
        # Make figure 
        f, axarr = plt.subplots(ncols=6, nrows=6, figsize=(24, 14), sharex=True)
        f.suptitle('RED: shelter to threat - BLUE: threat to shelter - DARKBLUE: threat to shelt excluding stim evoked')
        axarr = axarr.flatten()

        # Get tracking
        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess, sessions)
        speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)

        stimuli = get_session_stimuli_frames(mouse, sess)

        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    x_speed = derivative(tracking.x.values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            # Get the shelter->threat and threat-> shelter trips
            shelter_to_threat, threat_to_shelter = get_shelter_threat_trips(trace, mean_speed_th=speed_th)
            
            out_trips = pd.concat([trace[start:end] for start, end in shelter_to_threat], ignore_index=True).reset_index(drop=True)
            in_trips = pd.concat([trace[start:end] for start, end in threat_to_shelter], ignore_index=True).reset_index(drop=True)

            # Get only return trips that didn't follow the stimuli
            _, threat_to_shelter2 = get_shelter_threat_trips(trace, stimuli=stimuli, min_frames_after_stim=10*30)
            in_trips_no_stim = pd.concat([trace[start:end] for start, end in threat_to_shelter2], ignore_index=True).reset_index(drop=True)

            bins = np.linspace(400, 800, 9)
            dfs = dict(
                shelter_to_threat = (out_trips.groupby(pd.cut(out_trips['x'], bins=bins)), salmon),
                threat_to_shelter = (in_trips.groupby(pd.cut(in_trips['x'], bins=bins)), lightblue),
                # threat_to_shelter_no_stim = (in_trips_no_stim.groupby(pd.cut(in_trips_no_stim['x'], bins=bins)), blue)
            )

            # Plot lines for each ROI
            for val, (df, color) in dfs.items():
                int_values = np.array([(i.left + i.right)/2 for i in df.mean().index.values])
                
                if normalize_by_speed:
                    Y = df.mean().sig / df.mean().s
                else:
                    Y = df.mean().sig
                    axarr[nroi].fill_between(int_values, Y - df.std().sig, Y + df.std().sig, color=color, alpha=.2)
                
                axarr[nroi].plot(int_values, Y, lw=3, color=color, label=val) 


            if nroi == 0:
                # axarr[nroi].legend()
                print(f'\n{len(shelter_to_threat)} out and {len(threat_to_shelter)} in trips [{len(threat_to_shelter2)} with no stimuli]')
                axarr[-2].set(title = f'\n{len(shelter_to_threat)} out and {len(threat_to_shelter)} in trips [{len(threat_to_shelter2)} with no stimuli]')

            axarr[nroi].set(title=f'ROI - {nroi}')

        # Plot an histogram of the difference in speed between the two dataframes
        axarr[-1].bar(int_values, dfs['shelter_to_threat'][0].mean().s, color=salmon, 
                        alpha=.9, width=50,
                        label = 'shelter_to_threat')
        axarr[-1].bar(int_values, dfs['threat_to_shelter'][0].mean().s, color=lightblue, 
                        alpha=.3, width=50,
                        label = 'threat_to_shelter')
        # axarr[-1].bar(int_values, dfs['threat_to_shelter_no_stim'][0].mean().s, color=blue, 
        #                 alpha=.3, width=50,
        #                 label = 'threat_to_shelter_no_stim')
        axarr[-1].axhline(speed_th, lw=2, color=red)
        axarr[-1].set(title='Avg speed per bin', ylabel='speed', xlabel='X position')

            
        # Clean axes and save figure
        clean_axes(f)
        # f.tight_layout()
        save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\{mouse}_{sess}'+
                                f'_activity_vs_X_by_inout_trip_{"norm_by_speed" if normalize_by_speed else ""}', verbose=False)
        plt.close(f)
        # break
    # break



In [ ]:
df

# Speed
Plotting CA activity binned by speed

In [ ]:
# Plotting linear regression betwen speed and calcium activity
f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))



for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        axarr[n, s].set(title=f"{mouse} - {sess}")


        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
        # remove speed outliers
        speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)



        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]


            df = trace.groupby(pd.cut(trace['s'], bins=14)).mean()
            int_values = [(i.left + i.right)/2 for i in df.index.values]
            
            axarr[n, s].plot(int_values, df.sig, 'o-', lw=1, ms=3, label=f'roi {nroi}')

        # axarr[n, s].legend()
        # break
    # break
plt.show()


Plot speed and tracking overlay

In [ ]:

# Params
xbins, ybins = 25, 15
exploration_only = False

f, axarr = plt.subplots(ncols = 3, nrows=4, figsize=(16, 9))
f.suptitle(f"speed and tracking")

# Figure making
for m, mouse in tqdm(enumerate(mice)):
        for s, sess in enumerate(sessions[mouse]):
            tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
            # remove speed outliers
            speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)
            trace = pd.DataFrame(dict(
                x = np.int64(tracking['x'].values),
                y = np.int64(tracking['y'].values),
                speed = speed,
            ))

        
            # Plot tracking and binned activity
            axarr[m, s].hexbin(trace['x'], trace['y'], C=trace.speed, gridsize=xbins, reduce_C_function=np.mean)
            axarr[m, s].set(title=f'{mouse} - {sess}')

# Clean axes and save figure
clean_axes(f)
# f.tight_layout()
plt.show()

Bin frames in 5 groups based on speed thresholds and look at average ROI activity per Bin

In [ ]:
# f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))
f, ax = plt.subplots(figsize=(12, 10))
normalize = True
th = 5

px_to_cm = 13.9

endpoints = [[], [], [], [], []]
for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        # axarr[n, s].set(title=f"{mouse} - {sess}")


        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
        # remove speed outliers
        speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)



        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]

            stationary = trace.loc[trace.s < .1]
            slow = trace.loc[(trace.s > .1)&(trace.s < 1)]
            fast = trace.loc[trace.s > th] # trace.s.max()-1

            medium1 = trace.loc[(trace.s > 1)&(trace.s < th/2)]
            medium2 = trace.loc[(trace.s > th/2)&(trace.s < th)]


            y = np.array([stationary.sig.mean(), slow.sig.mean(), medium1.sig.mean(), medium2.sig.mean(), fast.sig.mean()])

            if normalize:
                y -= y[0]

            if not np.any(np.isnan(y[-1])):
                color = colorMap(y[-1], 'bwr', vmin=-100, vmax=100)
            else:
                color = colorMap(y[-2], 'bwr', vmin=-100, vmax=100)
            ax.plot([-1, 0, 1, 2, 3], y, 'o-', lw=2, ms=8, mec='k', label=f'roi {nroi}', color=color)
            # axarr[n, s]

            for k, yy in enumerate(y):
                if  not np.any(np.isnan(yy)):
                    endpoints[k].append(yy)


        # axarr[n, s].legend()
        # break
    # break
for k, endp in enumerate(endpoints):
    if k == 0: continue
    plot_kde(data=endp, vertical=True, normto=.4, z=k+.05-1, ax=ax,  color='gray', zorder=99)

cm_th = th*px_to_cm
ax.set(title='All ROIs activity binned by speed', xticks=[-1, 0, 1 ,2, 3], xticklabels=['stationary', '< 1 cm/frame', f'1<s<{round(cm_th/2, 2)}',
                f'{round(cm_th/2, 2)}<s<{cm_th}', f'> {cm_th} cm/frm'], xlabel='speed')

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\allrois_speed_modulation', verbose=False)
plt.show()


Do the same but splitting the data based on X position

In [ ]:
# f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))
f, axarr = plt.subplots(nrows=2, figsize=(12, 10))
normalize = True

th = 5
xth = 550


px_to_cm = 13.9

for i in [0, 1]:
    ax = axarr[i]

    
    endpoints = [[], [], [], [], []]
    for n, mouse in tqdm(enumerate(mice)):
        for s, sess in enumerate(sessions[mouse]):

            # axarr[n, s].set(title=f"{mouse} - {sess}")


            tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
            # remove speed outliers
            speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)



            for nroi in np.arange(_nrois):
                trace = pd.DataFrame(dict(
                        x = np.int64(tracking['x'].values),
                        y = np.int64(tracking['y'].values),
                        s =speed,
                        sig = signals[nroi],
                        isrec = is_rec,
                    )).interpolate()

                trace = trace.loc[trace.isrec > 0]

                if i == 0:
                    trace = trace.loc[trace.x < xth]
                else:
                    trace = trace.loc[trace.x > xth]

                stationary = trace.loc[trace.s < .1]
                slow = trace.loc[(trace.s > .1)&(trace.s < 1)]
                fast = trace.loc[trace.s > th] # trace.s.max()-1

                medium1 = trace.loc[(trace.s > 1)&(trace.s < th/2)]
                medium2 = trace.loc[(trace.s > th/2)&(trace.s < th)]


                y = np.array([stationary.sig.mean(), slow.sig.mean(), medium1.sig.mean(), medium2.sig.mean(), fast.sig.mean()])

                if normalize:
                    y -= y[0]

                if not np.any(np.isnan(y[-1])):
                    color = colorMap(y[-1], 'bwr', vmin=-100, vmax=100)
                else:
                    color = colorMap(y[-2], 'bwr', vmin=-100, vmax=100)
                ax.plot([-1, 0, 1, 2, 3], y, 'o-', lw=2, ms=8, mec='k', label=f'roi {nroi}', color=color)
                # axarr[n, s]

                for k, yy in enumerate(y):
                    if  not np.any(np.isnan(yy)):
                        endpoints[k].append(yy)


            # axarr[n, s].legend()
            # break
        # break
    for k, endp in enumerate(endpoints):
        if k == 0: continue
        plot_kde(data=endp, vertical=True, normto=.4, z=k+.05-1, ax=ax,  color='gray', zorder=99)

    cm_th = th*px_to_cm
    ax.set(title=f"All ROIs activity binned by speed {'left' if i == 0 else 'right'} half of arena", 
                        xticks=[-1, 0, 1 ,2, 3], xticklabels=['stationary', '< 1 cm/frame', f'1<s<{round(cm_th/2, 2)}',
                    f'{round(cm_th/2, 2)}<s<{cm_th}', f'> {cm_th} cm/frm'], xlabel='speed')

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\allrois_speed_xpos_modulation', verbose=False)
plt.show()


## Plotting tuning to angular velocity

In [ ]:
# f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))
f, ax = plt.subplots(figsize=(9, 6))
normalize = True

th = 0.25

endpoints = [[], [], []]
for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
    
        # Remove outliers
        ang_vel[ang_vel>np.nanpercentile(ang_vel, 99)] = np.nanpercentile(ang_vel, 99)
        ang_vel[ang_vel<np.nanpercentile(ang_vel, 1)] = np.nanpercentile(ang_vel, 1)



        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    avel = ang_vel,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]

            stationary = trace.loc[(trace.avel > -th)&(trace.avel<th)]
            cw = trace.loc[trace.avel > th]
            ccw = trace.loc[trace.avel < -th]
            


            y = np.array([ccw.sig.mean(), stationary.sig.mean(), cw.sig.mean()])

            if normalize:
                y -= y[1]

            if np.abs(np.nanmin(y)) > np.nanmax(y):
                color = colorMap(np.nanmin(y), 'bwr', vmin=-20, vmax=20)
                # color = 'gray'
                alpha = 1
            else:
                alpha = 1
                color = colorMap(np.nanmax(y), 'bwr', vmin=-20, vmax=20)

            # if not np.any(np.isnan(y[-1])):
            #     color = colorMap(y[-1], 'bwr', vmin=-100, vmax=100)
            # else:
            #     color = colorMap(y[-2], 'bwr', vmin=-100, vmax=100)
            ax.plot([-1, 0, 1], y, 'o-', lw=2, ms=8, mec='k', label=f'roi {nroi}', color=color, alpha=alpha)
            # axarr[n, s]

            for k, yy in enumerate(y):
                if not np.any(np.isnan(yy)):
                    endpoints[k].append(yy)



for k, endp in enumerate(endpoints):
    if k == 1: continue
    if k == 0:
        normto, z = -.4, -1.1
    elif k == 2:
        normto, z = .4, 1.1
    plot_kde(data=endp, vertical=True, normto=normto, z=z, ax=ax,  color='gray', zorder=99)

ax.set(title='All ROIs activity binned by speed', xticks=[-1, 0, 1], xticklabels=[f'< -{th} deg/frm', f'{-th}<avel<{th} deg/frm', f'> {th} deg/frm'],                          xlabel='angular velocity', ylabel='avg activity')

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\allrois_avel_modulation', verbose=False)
plt.show()


plot angular velocity over tracking

In [ ]:

# Params
xbins, ybins = 25, 15
exploration_only = False

f, axarr = plt.subplots(ncols = 3, nrows=4, figsize=(16, 9))
f.suptitle(f"speed and tracking")

# Figure making
for m, mouse in tqdm(enumerate(mice)):
        for s, sess in enumerate(sessions[mouse]):
            tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
            # Remove outliers
            ang_vel[ang_vel>np.nanpercentile(ang_vel, 99)] = np.nanpercentile(ang_vel, 99)
            ang_vel[ang_vel<np.nanpercentile(ang_vel, 1)] = np.nanpercentile(ang_vel, 1)

            trace = pd.DataFrame(dict(
                x = np.int64(tracking['x'].values),
                y = np.int64(tracking['y'].values),
                ang_vel = ang_vel,
            ))

        
            # Plot tracking and binned activity
            axarr[m, s].hexbin(trace['x'], trace['y'], C=trace.ang_vel, gridsize=xbins, reduce_C_function=np.mean)
            axarr[m, s].set(title=f'{mouse} - {sess}')

# Clean axes and save figure
clean_axes(f)
# f.tight_layout()
plt.show()

# Looking at stimuli evoked activity

In [ ]:
for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess, sessions)
        stimuli = get_session_stimuli_frames(mouse, sess)
        
        trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    avel = ang_vel,
                    isrec = is_rec,
                )).interpolate()
        
        for nroi in np.arange(_nrois):
            trace[f'{nroi}_sig']  = signals[nroi]

        f, ax = plt.subplots(figsize=(24, 12))

        for stim in stimuli:
            trial = trace[stim-30*10:stim+30*10]
            if np.any(trial.isrec.values == 0):
                raise ValueError('Recording was off during trial')

            ax.plot(trial.x.values[30*10:], trial.y.values[30*10:])
            ax.scatter(trial.x.values[30*10], trial.y.values[30*10], s=200, color=red)


        
        break
    break


